# Import Libraries

In [1]:
# importing libraries
import datetime
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.offline as pyo
import plotly.express as px
import collections
# must be installed manually
# conda install -c conda-forge pycountry
import pycountry
import difflib
from plotly.subplots import make_subplots
# statsmodels needs to be installed, used for trandlines

# Read CSV

In [2]:
# Suicides dataset: https://www.kaggle.com/russellyates88/suicide-rates-overview-1985-to-2016
# used to get suicides data
datset1_url = "https://stojkovski.ch/master.csv"
# Continent dataset: https://pkgstore.datahub.io/JohnSnowLabs/country-and-continent-codes-list/country-and-continent-codes-list-csv_csv/data/b7876b7f496677669644f3d1069d3121/country-and-continent-codes-list-csv_csv.csv
# used to get country continents
datset2_url = "https://stojkovski.ch/country_codes.csv"
# UV Ratiation in 2014 dataset: https://apps.who.int/gho/data/view.main.35300
# used to get UV Radiation in 2014
datset3_url = "https://stojkovski.ch/UV_1.csv"
# read the dataset and store it into a pandas dataframe
df = pd.read_csv(datset1_url)
df2 = pd.read_csv(datset2_url)
df3 = pd.read_csv(datset3_url)

In [3]:
df2

,Continent_Name,Continent_Code,Country_Name,Two_Letter_Country_Code,Three_Letter_Country_Code,Country_Number
0,Asia,AS,"Afghanistan, Islamic Republic of",AF,AFG,4.0
1,Europe,EU,"Albania, Republic of",AL,ALB,8.0
2,Antarctica,AN,Antarctica (the territory South of 60 deg S),AQ,ATA,10.0
3,Africa,AF,"Algeria, People's Democratic Republic of",DZ,DZA,12.0
4,Oceania,OC,American Samoa,AS,ASM,16.0
...,...,...,...,...,...,...
257,Africa,AF,"Zambia, Republic of",ZM,ZMB,894.0
258,Oceania,OC,Disputed Territory,XX,NaN,NaN
259,Asia,AS,Iraq-Saudi Arabia Neutral Zone,XE,NaN,NaN
260,Asia,AS,United Nations Neutral Zone,XD,NaN,NaN


# Data Cleaning

In [4]:
# TODO:
# 7 countries removed (<= 3 years of data total)
# 2016 data was removed (few countries had any, those that did often had data missing)
# HDI was removed due to 2/3 missing data
# Continent was added to the dataset using the countrycode package

# the following years are blacklisted beacuse the data contained incomplete or not accurate
blacklisted_years = [2016]
df = df.drop(df[df['year'].isin(blacklisted_years)].index)

# Data Wrangling

In [5]:
# Change wrong country names according to ISO 3166-1
df.loc[df['country'] == 'Czech Republic', 'country'] = 'Czechia'
df.loc[df['country'] == 'Republic of Korea', 'country'] = 'Korea, Republic of'
df.loc[df['country'] == 'Macau', 'country'] = 'Macao'

In [6]:
# Rename columns containing special characters
df.columns.values[9] = 'gdp_for_year_dollar'
df.columns.values[10] = 'gdp_per_capita_dollar'

In [7]:
# Remove comma
# old way
# df['gdp_for_year_dollar'].replace(',', '').astype(float)
# better way
# df['gdp_for_year_dollar'].apply(lambda x: x.replace(',', ''))

In [8]:
# Add country code
country_names = [x.name for x in pycountry.countries] 
best_match=dict()
for country in set(df['country']):
    best_confidence=0
    matching_countries = difflib.get_close_matches(country, country_names)
    for match in matching_countries:
        confidence = difflib.SequenceMatcher(None, match, country).ratio()
        if (confidence > 0.8 and confidence>best_confidence):
            best_confidence=confidence
            best_match[country]  = match
    if (matching_countries is None):
         best_match[country]  = 'Unknown'

best_matchh=dict()
for hey in best_match.keys():
    country1 = best_match.get(hey) 
    country2 = pycountry.countries.get(name=country1)
    best_matchh[hey]=country2.alpha_3
          
# for index, row in df.iterrows():   
#    df.loc[index, 'country code'] = best_matchh.get(df.loc[index, 'country'])
    
for country in set(df['country']):
    cc = best_matchh.get(country)
    df.loc[df['country']==country, 'country code'] = cc

In [9]:
# Add continent

# Slover version
# for index, row in df.iterrows():  
#     cc = df.loc[index, 'country code']
#     continent = df2[df2['Three_Letter_Country_Code'] == cc]['Continent_Name'].iloc[0]
#     df.loc[index, 'continent'] = continent

# Faster version
for country in set(df['country']):
    cc = df.loc[df['country']==country,'country code'].iloc[0]
    continent = df2[df2['Three_Letter_Country_Code'] == cc]['Continent_Name'].iloc[0]
    df.loc[df['country']==country, 'continent'] = continent
    

In [10]:
# suicides_no and population for each year
suicides_and_population_per_year = df.groupby("year")[['suicides_no','population']].sum()
# suicides/100k_pop every year
year_suicidesno_relation =  suicides_and_population_per_year["suicides_no"] / suicides_and_population_per_year["population"] * 100000
# mean of 100k_pop suicides
suicides_mean = year_suicidesno_relation.mean()

In [89]:

fig = go.Figure(data=go.Scatter(x=year_suicidesno_relation.index, y=year_suicidesno_relation.values, line=dict()))

# Horizontal Line (mean of suicides)
#fig.add_shape(
#        dict(
#            type="line",
#            x0=suicides_and_population_per_year.index.min(),
#            y0=suicides_mean,
#            x1=suicides_and_population_per_year.index.max(),
#            y1=suicides_mean,
#            line=dict(
#                color="MediumPurple",
#                width=4,
#                dash="dot",
#            )
#))

fig.update_layout(
    title=dict(
        text='Global suicides (per 100k people)',
        y=0.9,
        x=0.5,
        xanchor='center',
        yanchor='top'
    ),
    xaxis=dict(
        title='Year',
        linecolor='rgb(204, 204, 204)',
    ),
    yaxis=dict(
        title='Suicides per 100k',
    )
)

fig.show()


In [12]:
continent_df = df[['country','year','country code','continent','suicides/100k pop']]
continent_df = continent_df.groupby(['year','continent'])['suicides/100k pop'].mean().reset_index()
africa_df = continent_df[continent_df['continent']=='Africa']
asia_df = continent_df[continent_df['continent']=='Asia']
europe_df = continent_df[continent_df['continent']=='Europe']
namerica_df = continent_df[continent_df['continent']=='North America']
oceania_df = continent_df[continent_df['continent']=='Oceania']
samerica_df = continent_df[continent_df['continent']=='South America']

In [85]:
fig = make_subplots(rows=3,
                    cols=2,
                    subplot_titles=("Europe", "Asia", "Africa", "Oceania", "North America", "South America"),
                    shared_xaxes=True,
                    x_title="Year",
                    y_title="Suicide rate (per 100k people)",
                   )

fig.append_trace(go.Scatter(
    name="Europe",
    x=europe_df['year'],
    y=europe_df['suicides/100k pop'],
), row=1, col=1)

fig.append_trace(go.Scatter(
    name="Asia",
    x=asia_df['year'],
    y=asia_df['suicides/100k pop'],
), row=1, col=2)

fig.append_trace(go.Scatter(
    name="Africa",
    x=africa_df['year'],
    y=africa_df['suicides/100k pop'],
), row=2, col=1)

fig.append_trace(go.Scatter(
    name="Oceania",
    x=oceania_df['year'],
    y=oceania_df['suicides/100k pop'],
), row=2, col=2)

fig.append_trace(go.Scatter(
    name="North America",
    x=namerica_df['year'],
    y=namerica_df['suicides/100k pop'],
), row=3, col=1)

fig.append_trace(go.Scatter(
    name="South America",
    x=samerica_df['year'],
    y=samerica_df['suicides/100k pop'],
), row=3, col=2)


fig.update_layout(height=700, width=900,
        title=dict(
            text='Trends over time by Continent (per 100k people)',
            y=0.95,
            x=0.5,
            xanchor='center',
            yanchor='top'
        ),
        showlegend=False,
)

# Update xaxis properties
fig.update_xaxes(type="log",row=1, col=1)
fig.update_xaxes(type="log", row=1, col=2)
fig.update_xaxes(type="log", row=2, col=1)
fig.update_xaxes(type="log", row=2, col=2)
fig.update_xaxes(type="log", row=3, col=1)
fig.update_xaxes(type="log", row=3, col=2)

# Update yaxis properties
fig.update_yaxes(range=[0,europe_df['suicides/100k pop'].max()+5], nticks=6, row=1, col=1)
fig.update_yaxes(range=[0,asia_df['suicides/100k pop'].max()+5], nticks=6, row=1, col=2)
fig.update_yaxes(range=[0,africa_df['suicides/100k pop'].max()+5], nticks=6, row=2, col=1)
fig.update_yaxes(range=[0,oceania_df['suicides/100k pop'].max()+5], nticks=6, row=2, col=2)
fig.update_yaxes(range=[0,namerica_df['suicides/100k pop'].max()+5], nticks=6, row=3, col=1)
fig.update_yaxes(range=[0,samerica_df['suicides/100k pop'].max()+5], nticks=6, row=3, col=2)


fig.show()

In [86]:
suicides_per_country = df.groupby(['country'])['suicides/100k pop'].mean()
suicides_per_country.sort_values(ascending=False, inplace=True)

In [87]:
fig = go.Figure(go.Bar(
            x=suicides_per_country.values[10::-1],
            y=suicides_per_country.index[10::-1],
            orientation='h',
            text=suicides_per_country.values[10::-1]))
fig.update_layout(
    title=dict(
        text='Top 10 most impacted countries (per 100k people)',
        y=0.9,
        x=0.5,
        xanchor='center',
        yanchor='top'
    ),
    xaxis=dict(
        title='Suicides',
        linecolor='rgb(204, 204, 204)',
    ),
    yaxis=dict(
        title='Country',
    )
)
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()

In [88]:
fig = go.Figure(go.Bar(
            x=suicides_per_country.values[-10::1],
            y=suicides_per_country.index[-10::1],
            orientation='h',
            text=suicides_per_country.values[-10::1]))
fig.update_layout(
    title=dict(
        text='Top 10 least impacted countries (per 100k people)',
        y=0.9,
        x=0.5,
        xanchor='center',
        yanchor='top'
    ),
    xaxis=dict(
        title='Suicides',
        linecolor='rgb(204, 204, 204)',
    ),
    yaxis=dict(
        title='Country',
    )
)
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()

In [33]:
world_df = df[['country','year','country code','continent','suicides/100k pop']]
world_df = world_df.groupby(['country','country code','continent'])['suicides/100k pop'].mean().reset_index()

fig = px.scatter_geo(world_df, locations="country code", color="continent",
                     hover_name="country", size="suicides/100k pop",
                     projection="natural earth")
fig.update_layout(
    title=dict(
        text='Suicides Distribution on Map (per 100k people)',
        y=0.95,
        x=0.5,
        xanchor='center',
        yanchor='top'
    ),
)

fig.show()

In [19]:
sex_age_df = df.groupby(['age','sex'])['suicides/100k pop'].mean().reset_index()
sex_age_df.sort_values(ascending=True, inplace=True, by='suicides/100k pop')
males = sex_age_df.query('sex == "male"')
females = sex_age_df.query('sex == "female"')

sex_year_df = df.groupby(['sex','year'])['suicides/100k pop'].mean().reset_index()
males_year = sex_year_df.query('sex == "male"')
females_year = sex_year_df.query('sex == "female"')

In [20]:
fig = go.Figure(data=[
    go.Bar(name='Males', x=list(males['age']), y=males['suicides/100k pop'], text=males['suicides/100k pop']),
    go.Bar(name='Females', x=list(females['age']), y=females['suicides/100k pop'], text=females['suicides/100k pop'])
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.update_layout(
    title=dict(
        text='Age Distribution',
        y=0.90,
        x=0.5,
        xanchor='center',
        yanchor='top'
    ),
    xaxis=dict(
        title='Age Range',
        linecolor='rgb(204, 204, 204)',
    ),
    yaxis=dict(
        title='Suicides (per 100k people)',
    )
)
fig.show()

In [21]:
sex_df=df.groupby(['sex'])['suicides/100k pop'].mean().reset_index()

In [22]:
fig = px.pie(sex_df, values='suicides/100k pop', names='sex',
    title=dict(
        text='Gender Distribution',
        y=0.90,
        x=0.5,
        xanchor='center',
        yanchor='top'
    ),
             hover_data=['suicides/100k pop'])
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [23]:
scatter = df.groupby(['country','continent'])[['gdp_per_capita_dollar', 'suicides/100k pop']].mean().reset_index()

In [24]:
fig = px.scatter(scatter, x="gdp_per_capita_dollar", y="suicides/100k pop", trendline='ols')
fig.show()

In [25]:
hdi_not_null_df = df[df['HDI for year'].notnull()]
scatter2 = hdi_not_null_df.groupby(['country','continent'])[['HDI for year', 'suicides/100k pop']].mean().reset_index()

In [26]:
fig = px.scatter(scatter2, x="HDI for year", y="suicides/100k pop", color='continent',trendline='ols')
fig.show()

In [27]:
fig = px.scatter(scatter2, x="HDI for year", y="suicides/100k pop",trendline='ols')
fig.show()

In [28]:
only_2004 = df[df['year'] > 2004].copy()
for country in only_2004['country'].unique():
    if df3[df3['Country'] == country]['Country'].any():
        uv_radiation_for_country = df3[df3['Country'] == country]['UV radiation'].iloc[0]
        only_2004.loc[only_2004['country'] == country, 'UV radiation'] = uv_radiation_for_country

only_2004_uv_not_null = only_2004.loc[only_2004['UV radiation'].notnull()].copy()

In [29]:
scatter3 = only_2004.groupby(['country','continent','UV radiation'])[['suicides/100k pop']].mean().reset_index()

In [35]:
fig = px.scatter(scatter3, x="UV radiation", y="suicides/100k pop", trendline='ols')

fig.update_layout(
    title=dict(
        text='Correletion between UV Radiation and Suicides',
        y=0.95,
        x=0.5,
        xanchor='center',
        yanchor='top'
    ),
)

fig.show()

In [31]:
fig = make_subplots(
	rows=2,
    cols=2,
    specs=[[{"type": "domain","rowspan": 2}, {}],[{}, {}]],
    shared_xaxes=True,
)    

fig.add_trace(go.Pie(
    values=sex_df['suicides/100k pop'],
    labels=sex_df['sex'],
    textinfo='label+percent',
    hole=.4
),row=1, col=1)


fig.add_trace(go.Scatter(
    name="Males",
    x=males_year['year'],
    y=males_year['suicides/100k pop'],
    line=dict(color='blue'),
    showlegend=False
), row=1, col=2)


fig.add_trace(go.Scatter(
    name="Females",
    x=females_year['year'],
    y=females_year['suicides/100k pop'],
    line=dict(color='rgb(241, 105, 19)'),
    showlegend=False
), row=2, col=2)

fig.update_xaxes( type="log",row=1, col=2)
fig.update_xaxes( type="log", row=2, col=2)


fig.update_layout(height=500, width=900,
        title=dict(
            text='Gender Distribution and Trend (per 100k people)',
            y=0.90,
            x=0.5,
            xanchor='center',
            yanchor='top'
        ),
)
fig.show()

In [39]:
samerica_df['suicides/100k pop'].max()

15.219907407407405